In [1]:
import cv2
import time
import numpy as np

from mss import mss

In [23]:
def pathfinding(img):
    threshold = 0
    cells, head_pos = get_cells(img, Label, threshold)
    
    if(head_pos is None):
        return None
    
    cm_pos = center_of_mass(cells, nrows, ncols)
#     min_body_pos = find_min_max_body_pos(cells, nrows, ncols, True)
#     max_body_pos = find_min_max_body_pos(cells, nrows, ncols, False)

    new_cells = find_optimal_path(cells, head_pos, [cm_pos], [1])
    if(new_cells is None):
        return None

    print(new_cells)
    print("=============================")
    
    return new_cells

In [3]:
# detect game over
def game_over(img):
    (left, top, right, bottom) = (453, 436, 533, 515)
    img_crp = img[top:bottom, left:right]
    
    img_score_path = 'img_raw/gameover.png'
    img_score_full = cv2.imread(img_score_path)
    img_score = img_score_full[top:bottom, left:right]
    
    hist_crp = calc_hist(img_crp)
    hist_score = calc_hist(img_score)
    
    sum_diff = 0
    for chan in range(3):
        diff = cv2.compareHist(hist_crp[chan], hist_score[chan], cv2.HISTCMP_CORREL)
        sum_diff += diff
    
    avg_diff = sum_diff/3
    if(avg_diff>0.9):
        return True
    
    return False

In [5]:
def screenshot():
    field_width = 688
    field_height = 608
    field_left = 607
    field_top = 302
    
    start_time = time.time()
    with mss() as sct:        
        monitor_number = 1
        mon = sct.monitors[monitor_number]
        monitor = {'mon': monitor_number, 'width': field_width, 'height': field_height, 'left': mon["left"] + field_left, 'top': mon["top"] + field_top}

        sct_img = sct.grab(monitor)
        img = np.array(sct_img.pixels).astype(np.uint8)[::,::,::-1]
        
        end_time = time.time()
        duration = end_time - start_time
#         print(duration)
        
        return img

In [6]:
def main():    
    while(True):
        start_time = time.time()
        img_ret = screenshot()
        new_cells = pathfinding(img_ret)

        gameover = game_over(img_ret)
        if(gameover):
            print("game over")
            break
            
        duration = time.time() - start_time
        print(duration)

In [20]:
%run Pathfinding.ipynb
%run astar.ipynb

In [24]:
main()

0.4346287250518799
0.4382610321044922
0.4490072727203369
0.4480159282684326
0.43164753913879395
0.4321327209472656
0.43988633155822754
0.453869104385376
0.45415711402893066
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  3 -1 -1 -1 -1 -1 -1 -1 -1  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
0.45758676528930

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
0.4470088481903076
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0  0  0  0  0  0  0  2  0  0]
 [ 0  0  0  0 -1  0  0  0  0  0  0  0  0  0  2  0  0]
 [ 0  0  0  0 -1  0  0  0  0  0  0  0  0  0  2 

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [-1  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [-1  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
0.45211172103881836
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [3 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 -1 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  2  2  2 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  3 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
0.4478616714477539
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 -1  1  1  0  0  0  0  0  0  0  0  0]
 [ 0  0 